# 0.2.0 Simple topic identification

In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
from matplotlib import pyplot as plt
import re

from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize, TweetTokenizer
from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [7]:
import sys
sys.path.append("../") 

from utils.info import article
import utils.paths as path
from utils.paths2 import direcciones, direcciones_cursos

## 0.1.2 Bag-of-words picker

In [5]:
Counter(word_tokenize("""The cat is in the box. The cat box."""))

Counter({'The': 2, 'cat': 2, 'is': 1, 'in': 1, 'the': 1, 'box': 2, '.': 2})

## 0.1.3 Building a Counter with bag-of-words

In [33]:
print('ok_')

ok_
